In [46]:
import pandas as pd
import numpy as np
import sqlite3

import pyodbc
DB = {'servername': 'localhost\SQLExpress',
      'database': 'testDB'}# create the connection
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
df = pd.read_sql_query('SELECT * FROM TestTable',conn)

In [60]:
def load_db(path):
    conn = sqlite3.connect(path)
    return conn

def get_tables(conn):
    return pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

def drop_trial_column(table):
    # Search for Column Like %TRIAL%
    trial_columns = [col for col in table.columns if 'TRIAL' in col]
    # Drop Columns
    table = table.drop(columns=trial_columns)
    return table

def load_table(conn, table):
    query = "SELECT * FROM {}".format(table)
    df = pd.read_sql_query(query, conn)
    return drop_trial_column(df)

def print_columns(frame: pd.DataFrame):
    for col in frame.columns:
        print(col)


In [61]:
go_crm = load_db('source/go_crm.sqlite')

tables_crm = get_tables(go_crm)
df = pd.read_sql_query("SELECT * FROM retailer_segment", go_crm)

df

,SEGMENT_CODE,LANGUAGE,SEGMENT_NAME,SEGMENT_DESCRIPTION,TRIAL222
0,1,EN,Outdoors Shop - Exclusive,"Primarily offers camping supplies, hiking equi...",T
1,10,EN,Eyewear Store,Primarily offers prescription and fashion eyew...,T
2,11,EN,Sports Store - Exclusive,Primarily offers a wide range of sporting good...,T
3,12,EN,Sports Store,Primarily offers a wide range of sporting goods.,T
4,2,EN,Outdoors Shop,"Primarily offers camping supplies, hiking equi...",T
5,3,EN,Golf Shop,Primarily offers golf equipment and accessories.,T
6,4,EN,Department Store - Exclusive,Offers a wide range of goods. Concentrates on...,T
7,5,EN,Department Store,Offers a wide range of goods.,T
8,6,EN,Direct Marketing - Exclusive,Offers a wide range of goods through mail orde...,T
9,7,EN,Direct Marketing,Offers a wide range of goods through mail orde...,T


In [87]:
retailer_heaquarters = load_table(go_crm, 'retailer_headquarters').add_prefix('RETAILER_HEADQUARTERS_')
retailer_site = load_table(go_crm, 'retailer_site').add_prefix('RETAILER_SITE_')
retailer_segment = load_table(go_crm, 'retailer_segment').add_prefix('RETAILER_SEGMENT_')
retailer_country = load_table(go_crm, 'country').add_prefix('RETAILER_COUNTRY_')
sales_demographic = load_table(go_crm, 'sales_demographic').add_prefix('SALES_DEMOGRAPHIC_')
sales_territory = load_table(go_crm, 'sales_territory').add_prefix('SALES_TERRITORY_')
age_group = load_table(go_crm, 'age_group').add_prefix('AGE_GROUP_')


result = pd.merge(
    sales_demographic, retailer_heaquarters, left_on="SALES_DEMOGRAPHIC_RETAILER_CODEMR", right_on="RETAILER_HEADQUARTERS_RETAILER_CODEMR"
    ).drop(columns='SALES_DEMOGRAPHIC_RETAILER_CODEMR')

result = pd.merge(
    result, retailer_segment, left_on='RETAILER_HEADQUARTERS_SEGMENT_CODE', right_on='RETAILER_SEGMENT_SEGMENT_CODE'
    ).drop(columns='RETAILER_HEADQUARTERS_SEGMENT_CODE')

result = pd.merge(
    result, age_group, left_on="SALES_DEMOGRAPHIC_AGE_GROUP_CODE", right_on="AGE_GROUP_AGE_GROUP_CODE"
    ).drop(columns='SALES_DEMOGRAPHIC_AGE_GROUP_CODE')

result = pd.merge(
    result, retailer_country, left_on='RETAILER_HEADQUARTERS_COUNTRY_CODE', right_on='RETAILER_COUNTRY_COUNTRY_CODE'
    ).drop(columns='RETAILER_HEADQUARTERS_COUNTRY_CODE')

result = pd.merge(
    result, sales_territory, left_on='RETAILER_COUNTRY_SALES_TERRITORY_CODE', right_on='SALES_TERRITORY_SALES_TERRITORY_CODE'
    ).drop(columns='RETAILER_COUNTRY_SALES_TERRITORY_CODE')




SALES_DEMOGRAPHIC_DEMOGRAPHIC_CODE
SALES_DEMOGRAPHIC_SALES_PERCENT
RETAILER_HEADQUARTERS_RETAILER_CODEMR
RETAILER_HEADQUARTERS_RETAILER_NAME
RETAILER_HEADQUARTERS_ADDRESS1
RETAILER_HEADQUARTERS_ADDRESS2
RETAILER_HEADQUARTERS_CITY
RETAILER_HEADQUARTERS_REGION
RETAILER_HEADQUARTERS_POSTAL_ZONE
RETAILER_HEADQUARTERS_PHONE
RETAILER_HEADQUARTERS_FAX
RETAILER_SEGMENT_SEGMENT_CODE
RETAILER_SEGMENT_LANGUAGE
RETAILER_SEGMENT_SEGMENT_NAME
RETAILER_SEGMENT_SEGMENT_DESCRIPTION
AGE_GROUP_AGE_GROUP_CODE
AGE_GROUP_UPPER_AGE
AGE_GROUP_LOWER_AGE
RETAILER_COUNTRY_COUNTRY_CODE
RETAILER_COUNTRY_COUNTRY_EN
RETAILER_COUNTRY_FLAG_IMAGE
SALES_TERRITORY_SALES_TERRITORY_CODE
SALES_TERRITORY_TERRITORY_NAME_EN


In [50]:

# Resulting tables colums:

# SALES_DEMOGRAPHIC_code
# SALES_DEMOGRAPHIC_sales_percent

# AGE_GROUP_code
# AGE_GROUP_lower_age
# AGE_GROUP_upper_age

# RETAILER_SEGMENT_code
# RETAILER_SEGMENT_language
# RETAILER_SEGMENT_name
# RETAILER_SEGMENT_description

# RETAILER_HEADQUARTERS_codemmr
# RETAILER_HEADQUARTERS_retailer_name
# RETAILER_HEADQUARTERS_address1
# RETAILER_HEADQUARTERS_address2
# RETAILER_HEADQUARTERS_city
# RETAILER_HEADQUARTERS_region
# RETAILER_HEADQUARTERS_postal_code
# RETAILER_HEADQUARTERS_phone
# RETAILER_HEADQUARTERS_fax

# RETAILER_COUNTRY_code
# RETAILER_COUNTRY_name_en
# RETAILER_COUNTRY_flag_image

# SALES_TERRITORY_code
# SALES_TERRITORY_name_en